In [68]:
ip='172.31.19.202'
db='amazon_sp_api'
password='Techblooprint123'
user="blooprint"


In [69]:
from pyspark.sql import SparkSession
import os
from clickhouse_driver import Client

spark = SparkSession.builder \
    .appName("BigCSVWrite") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "50")
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F
# Should not raise:
spark.sparkContext._jvm.java.lang.Class.forName("org.postgresql.Driver")

URL  = os.getenv("PG_JDBC_URL", f"jdbc:postgresql://{ip}:5433/{db}")
USER = os.getenv("PG_USER", user)
PWD  = os.getenv("PG_PASSWORD", password)
portfolio_query = "(SELECT * FROM public.portfolio_id_name_mapping) AS p"

portfolio_id_name_mapping = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", portfolio_query)
      .load())



In [70]:
sp_advertised_query = """
(SELECT account_name,
            "advertisedAsin" AS asin,
            "campaignId" AS campaign_id,
            "campaignName" AS campaign_name,
            "adGroupName" AS ad_group_name,
            date AS date,
            impressions,
            clicks,
            spend AS ads_spend,
            sales30d AS ads_sales,
            "unitsSoldClicks30d" AS units
           FROM public.sp_advertised_product) as ad_p

"""

sp_advertised_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sp_advertised_query)
      .load())
sp_advertised_df=sp_advertised_df.withColumn("date",
    F.col("date").cast(T.DateType())
)

In [71]:
sd_advertised_query = """
(SELECT account_name,
            "promotedAsin" AS asin,
            "campaignId" AS campaign_id,
            "campaignName" AS campaign_name,
            "adGroupName" AS ad_group_name,
            date AS date,
            impressions,
            clicks,
            cost AS ads_spend,
            sales AS ads_sales,
            "unitsSold" AS units
           FROM public.sd_advertised_product) as sd_advertised_query
"""

sd_advertised_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sd_advertised_query)
      .load())
sd_advertised_df=sd_advertised_df.withColumn("date",
    F.col("date").cast(T.DateType())
)

In [72]:
sb_advertised_query = """
(select account_name,
            "purchasedAsin" AS asin,
            "campaignId" AS campaign_id,
            "campaignName" AS campaign_name,
            "adGroupName" AS ad_group_name,
            date AS date,
            NULL::numeric AS impressions,
            NULL::numeric AS clicks,
            NULL::numeric AS ads_spend,
            sales14d AS ads_sales,
            "unitsSold14d" AS units from public.sb_purchased_product) as sb_advertised_query
"""

sb_advertised_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sb_advertised_query)
      .load())
sb_advertised_df=sd_advertised_df.withColumn("date",
    F.col("date").cast(T.DateType())
)

In [73]:
advertised_asin=sp_advertised_df.unionByName(sb_advertised_df).unionByName(sd_advertised_df)

In [74]:
advertised_asin=advertised_asin.withColumn("campaign_id",
                           F.split(
                               F.col("campaign_id").cast(T.StringType()),r'\.'
                               )[0].astype("long")
                            )

In [75]:
campaign_query = "(SELECT * FROM public.campaigns) as c"
campaigns = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", campaign_query)
      .load())

campaign_df=campaigns.select(F.col("campaignId").alias("campaign_id"),
                 F.col("portfolioId").alias("portfolioId")
                 ).dropDuplicates(['campaign_id'])
portfolio_df=portfolio_id_name_mapping.select(F.col("portfolioId").alias("portfolioId"),
                                 F.col("name").alias("portfolio_name")).dropDuplicates(['portfolioId'])
portfolio_mapping=campaign_df.join(portfolio_df,on='portfolioId',how='left')

In [77]:
portfolio_mapping.show()

+---------------+---------------+------------------+
|    portfolioId|    campaign_id|    portfolio_name|
+---------------+---------------+------------------+
|           NULL|100076249959544|              NULL|
|227942005450585|100118912681214|     Collagen Skin|
|           NULL|100331592418431|              NULL|
|           NULL|100373123647857|              NULL|
|           NULL|  1004821708244|              NULL|
|199478083998011|100645553381123|     Collagen Bone|
|           NULL|100688068888370|              NULL|
| 12236653894609|100725861019934|         Audio ANS|
|           NULL|100790136138793|              NULL|
| 97769239140773| 10118589023770|      Whey Protein|
|231025265874213|101199648918799|        Moon Latte|
|           NULL|101242142228603|              NULL|
|           NULL|101260849427507|              NULL|
|258918003083195| 10129075998155|             GLASS|
|109044300472709|101411534730219|MultiVitamin Women|
|           NULL|101429469803822|             

In [79]:
portfolio_mapping=portfolio_mapping.drop_duplicates()

In [ ]:
asin_level_data=portfolio_mapping.join(advertised_asin,how='outer',on="campaign_id")

In [81]:
asin_data=asin_level_data.fillna({"impressions": 0,
                "clicks":0,
                 "ads_spend":0,
                 "UNITS":0,
                 "ads_sales":0,
                 "units":0,
                }).select(
                    F.col("account_name").alias("ACCOUNT NAME"),
                    F.col("asin").alias("ASINS"),
                    F.col("portfolio_name").alias("PORTFOLIOS"),
                    F.col("campaign_name").alias("CAMPAIGNS"),
                    F.col("ad_group_name").alias("AD GRP"),

                    F.col("date").alias("DATE"),
                    F.col("impressions").astype("int").alias("IMPRSN"),
                    F.col("clicks").astype("int").alias("CLICKS"),
                    F.col("ads_spend").astype("int").alias("SPEND"),
                    F.col("ads_sales").astype("int").alias("SALES"),
                    F.col("units").astype("int").alias("UNITS")

                )

In [82]:
dummy_asin_data=asin_data.filter(F.col("ACCOUNT NAME")=='awenest')

In [83]:
pattern = r"(?i)\bawenest\b"   
string_cols = [c for c, t in dummy_asin_data.dtypes if t == "string"]

dummy_asin_data = dummy_asin_data.select([
    F.regexp_replace(F.col(c), pattern, "Dummy").alias(c) if c in string_cols else F.col(c)
    for c, _ in dummy_asin_data.dtypes
])

In [84]:
final_asin_data=asin_data.unionByName(dummy_asin_data)

In [85]:
(
    final_asin_data.write
    .format("jdbc")
    .option("url", URL)
    .option("user", USER)
    .option("password", PWD)
    .option("driver", "org.postgresql.Driver")
    .option("dbtable", "public.asin_info")  # destination table
    .mode("overwrite")  # creates table if not exists, replaces if exists
    .save()
)
